In [5]:
from bokeh.io import curdoc, show, output_notebook, push_notebook
from bokeh.layouts import column, widgetbox, row, gridplot
from bokeh.models import ColumnDataSource, Slider, Button, TextInput, Select, Dropdown, CheckboxGroup
from bokeh.plotting import figure

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
output_notebook()

Loading BokehJS ...

In [6]:
t= np.arange(0,5000,50) #t outside function
x0=[0.1,0.1,0,0]

#first order plot
# A --> B (k1)
# B --> C (k2)
#initial conc A0 = 0.1, B0 = 0, C0 = 0
#k1 = 0.01, k2 = 0.005

#differential equations: dA/dt = -k1[A], dB/dt = k1[A]-k2[B], dC/dt = k2[B]
#x[0] == A, x[1] = B, x[2] == C

def f(k1,k2,initial): #function to return the numpy array of [A,B,C] concentrations
    def rxn(initial,t): #function for the differential equations
        r1=k1*initial[0]
        r2=k2*initial[1]
        
        dAdt=-r1
        dBdt=r1-r2
        dCdt=r2
        
        return [dAdt, dBdt,dCdt]
        
    Conc = odeint(rxn, initial,t)
    
    return Conc

#Conc=f(0.01,0.005,[0.1,0,0]) #Assign variable conc to resutls from function, f(initial conditions)

#A=Conc[:,0]
#B=Conc[:,1]
#C=Conc[:,2]

#plt.plot(t,A)
#plt.plot(t,B)
#plt.plot(t,C)


    
    
    
    

In [7]:
#A + B --> C (k1)
# C --> D (k2)
#initial conc A0 = 0.1, B0 = 0.1, C0 = 0, D0 = 0
#k1 = 0.01, k2 = 0.005

#differential equations: dA/dt = -k1[A][B] = dB/dt, dC/dt = k1[A][B]-k2[C], dD/dt = k2[C]
#x[0] == A, x[1] == B, x[2] == C, x[3] == D

def f2(k1,k2,initial):
    def AB(initial,t):
        r1=k1*initial[0]*initial[1]
        r2=k2*initial[2]
        
        dAdt = -r1
        dBdt = -r1
        dCdt = r1-r2
        dDdt = r2
        
        return [dAdt,dBdt,dCdt,dDdt]
    
    Conc = odeint(AB, initial, t)
    return Conc

#Conc = f2(0.01, 0.005, x0)

#A=Conc[:,0]
#B=Conc[:,1]
#C=Conc[:,2]
#D=Conc[:,3]


#plt.plot(t,A, color='red')
#plt.plot(t,B, color= 'blue')
#plt.plot(t,C, color = 'black')
#plt.plot(t,D, color = 'green')


        
        


In [11]:
# Generating array for each set of reactions
Conc1 = f(0.01, 0.005, [0.1, 0, 0])
Conc2 = f2(0.01,0.005,[0.1, 0.1, 0, 0]) 

#Separating the array into individual columns
A1=Conc1[:,0]
B1=Conc1[:,0]
C1=Conc1[:,0]

A2=Conc2[:,0]
B2=Conc2[:,1]
C2=Conc2[:,2]
D2=Conc2[:,3]

#Defining initial plot
source1 = ColumnDataSource(data = {'t': t,'A1': A1})
source2 = ColumnDataSource(data = {'t': t,'B1': B1})
source3 = ColumnDataSource(data = {'t': t,'C1': C1})

source4 = ColumnDataSource(data = {'t':t, 'A2': A2})
source5 = ColumnDataSource(data = {'t':t, 'B2': B2})
source6 = ColumnDataSource(data = {'t':t, 'C2': C2})
source7 = ColumnDataSource(data = {'t':t, 'D2': D2})

#Creating figures
p1=figure(x_axis_label = 'Time / s', y_axis_label = 'Conc / M')
p2=figure(x_axis_label = 'Time / s', y_axis_label = 'Conc / M')

#Creating lines
p1.line('t','A1', source=source1, color = 'red')
p1.line('t','B1', source=source2, color = 'black')
p1.line('t','C1', source=source3, color = 'green')


p2.line('t','A2', source=source4, color = 'red')
p2.line('t','B2', source=source5, color = 'blue')
p2.line('t','C2', source=source6, color = 'black')
p2.line('t','D2', source=source7, color = 'green')

#Creating widgets
slider1a = Slider(start = 0.0001, end = 0.05, step = 0.001, value = 0.01, title = 'k1')
slider1b = Slider(start = 0.0001, end = 0.01, step = 0.0001, value = 0.005, title = 'k2')
input1a = TextInput(value = '0.1', title = '[A]0')

slider2a = Slider(start = 0.0001, end = 0.05, step = 0.001, value = 0.01, title = 'k1')
slider2b = Slider(start = 0.0001, end = 0.05, step = 0.001, value = 0.01, title = '2')
input2a = TextInput(value = '0.1', title = '[A]0')
input2b = TextInput(value = '0.1', title = '[B]0')

def callback(attr, old, new):
    k1a=slider1a.value
    k1b=slider1b.value
    A10=float(input1a.value)
        
    k2a=slider2a.value
    k2b=slider2b.value
    A20=float(input2a.value)
    B20=float(input2b.value)
    
    new_conc1=f(k1a,k1b,[A10,0,0])
    
    new_A1=new_conc1[:,0]
    new_B1=new_conc1[:,1]
    new_C1=new_conc1[:,2]
    
    source1.data={'t':t,'A1':new_A1}
    source2.data={'t':t,'B1':new_B1}
    source3.data={'t':t,'C1':new_C1}
      
    new_conc2=f2(k2a,k2b,[A20,B20,0,0])
    
    new_A2=new_conc2[:,0]
    new_B2=new_conc2[:,1]
    new_C2=new_conc2[:,2]  
    new_D2=new_conc2[:,3]

    source4.data={'t':t,'A2':new_A2}
    source5.data={'t':t,'B2':new_B2}
    source6.data={'t':t,'C2':new_C2}
    source7.data={'t':t,'D2':new_D2}
    
    push_notebook()

      
input1a.on_change('value', callback)
slider1a.on_change('value', callback)
slider1b.on_change('value', callback)
    

slider2a.on_change('value', callback)
slider2b.on_change('value', callback)
input2a.on_change('value', callback)
input2b.on_change('value', callback)

layout=gridplot([[widgetbox(slider1a, slider1b, input1a), widgetbox(slider2a, slider2b, input2a,input2b)],[p1,p2]])


#layout = row(column(widgetbox(slider1a, slider1b, input1a) slider2, input1, input2, dropdown)),p1,p2)

curdoc().add_root(layout)